### Library import and code configuration

In [1]:
import numpy as np
import pandas as pd
import geocoder

In [27]:
pd.options.display.float_format = '{:.6f}'.format

Geocoder

In [3]:
def geoc(loc):
    g = geocoder.google(loc)
    g = g.json
    print (loc)
    if len(g) != 2 or g is None:
        g = geocoder.arcgis(loc)
        g = g.json

        if len(g) != 2 or g is None:
            g = geocoder.geocodefarm(loc)
            g = g.json

            if len(g) != 2 or g is None:
                g = geocoder.osm(loc)
                g = g.json

                if len(g) != 2 or g is None:
                    g = geocoder.yandex(loc)
                    g = g.json
    else:
        g = None
    
    print(g)
    return g

In [4]:
def getAirlines(txt):
    airlines = ['aena','aeromar','aeromexico','aireuropa','airmadrid','airnostrum','americanairlines',
                       'avianca','blueair','britishairways','easyjet','emirates','eurowings','flybe','iberia','klm',
                       'lufthansa','niki','norwegian','qatar','ryanair','spanair','spiritairlines','tame','vasp',
                       'volaris','vueling','westjet','wizzair']
    
    arl = ''
    
    for airline in airlines:
        if airline.lower() in txt.lower():
            arl = airline
        
    return arl

### Dataframe

In [5]:
fileIn = '/media/dedsresende/BackUpDeds/Github_UB/CapstoneProject/Feelings/kagle_capstone_project/presentacion/data/tweets_public_decode.csv'
fileOut = '/media/dedsresende/BackUpDeds/Github_UB/CapstoneProject/Feelings/kagle_capstone_project/presentacion/data/tweets_public_clean.csv'

In [6]:
df = pd.read_csv(fileIn, encoding = 'utf-16')
df.head()

,airline_sentiment,is_reply,reply_count,retweet_count,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,positive,False,0,0,"""Los pilotos de Ryanair desconvocan la huelga ...",NaN,Mon Dec 18 13:07:04 +0000 2017,942743012337122944.00,NaN,Dublin
1,positive,True,0,0,"@Iberia @lavecinarubia Si ,por favor las decla...",NaN,Sat Nov 04 17:05:11 +0000 2017,926857871916184064.00,NaN,NaN
2,neutral,True,0,0,@Iberia Me dirías por favor que costo tiene?,NaN,Sat Dec 02 15:24:09 +0000 2017,936979305720091008.00,NaN,NaN
3,negative,True,0,0,"@SupermanlopezN @Iberia @giroditalia Champion,...",NaN,Thu Dec 21 23:17:43 +0000 2017,943983853802328064.00,NaN,Central Time (US & Canada)
4,negative,True,0,0,@SrtaFarrellDM @KLM @Iberia Eso de avianca es ...,NaN,Wed Dec 06 00:44:25 +0000 2017,938207464457212032.00,NaN,Eastern Time (US & Canada)


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7867 entries, 0 to 7866
Data columns (total 10 columns):
airline_sentiment    7867 non-null object
is_reply             7867 non-null bool
reply_count          7867 non-null int64
retweet_count        7867 non-null int64
text                 7867 non-null object
tweet_coord          22 non-null object
tweet_created        7867 non-null object
tweet_id             7867 non-null float64
tweet_location       439 non-null object
user_timezone        5119 non-null object
dtypes: bool(1), float64(1), int64(2), object(6)
memory usage: 560.9+ KB


In [8]:
df['tweet_id'] = df['tweet_id'].astype('category')
df['airline_sentiment'] = df['airline_sentiment'].astype('category')
df['retweet_count'] = df['retweet_count'].astype('int64')
df['text'] = df['text'].astype('str')
df['tweet_coord'] = df['tweet_coord'].astype('str')
df['tweet_created'] = pd.to_datetime(df['tweet_created'])
df['tweet_location'] = df['tweet_location'].astype('category')
df['user_timezone'] = df['user_timezone'].astype('category')

In [9]:
df['lat'] = df['tweet_coord'].apply(lambda x: np.nan if x == 'nan' else float((x.replace('[',' ').replace(']',' ').replace(' ','').split(','))[0]))
df['lon'] = df['tweet_coord'].apply(lambda x: np.nan if x == 'nan' else float((x.replace('[',' ').replace(']',' ').replace(' ','').split(','))[1]))

In [10]:
df['airline'] = df['text'].apply(lambda x: getAirlines(x))

In [11]:
ttloc = {}

for loc in df[df['tweet_location'].notnull()]['tweet_location'].unique():
    g = geocoder.google(loc.split(',')[0])
    latlng = g.latlng
    
    if latlng is None:
        g = geocoder.arcgis(loc.split(',')[0])
        latlng = g.latlng

        if latlng is None:
            g = geocoder.geocodefarm(loc.split(',')[0])
            latlng = g.latlng

            if latlng is None:
                g = geocoder.osm(loc.split(',')[0])
                latlng = g.latlng

                if latlng is None:
                    g = geocoder.yandex(loc.split(',')[0])
                    latlng = g.latlng
    
    try:
        ttloc[loc] = {'lat': latlng[0], 'lng': latlng[1]}
        print(ttloc[loc])
        
    except Exception as e:
        print(e)

{'lat': 40.4167754, 'lng': -3.7037902}
{'lat': -34.459329999999966, 'lng': -58.91528999999997}
{'lat': 1.2058837, 'lng': -77.285787}
{'lat': 41.6488226, 'lng': -0.8890853}
{'lat': 40.6400629, 'lng': 22.9444191}
{'lat': 40.4167754, 'lng': -3.7037902}
{'lat': 43.529742, 'lng': 5.447426999999999}
{'lat': 19.4306069, 'lng': -99.0949899}
{'lat': 36.329503, 'lng': -94.1159519}
{'lat': -33.3989812, 'lng': -70.5573124}
{'lat': 40.42028000000005, 'lng': -3.7057699999999727}
{'lat': 38.90482859999999, 'lng': -77.05239870000001}
{'lat': 43.3069814, 'lng': -2.9126022}
{'lat': 36.2786587, 'lng': -94.30429400000001}
{'lat': -34.6036844, 'lng': -58.3815591}
{'lat': 41.3120158, 'lng': 2.0959429}
{'lat': -1.831239, 'lng': -78.18340599999999}
{'lat': 50.1109221, 'lng': 8.6821267}
{'lat': 41.3850639, 'lng': 2.1734035}
{'lat': 10.3910485, 'lng': -75.4794257}
{'lat': 41.10934890000001, 'lng': 1.1400998}
{'lat': 40.5688791, 'lng': -3.5913939}
{'lat': 19.3631419, 'lng': -99.28805969999999}
{'lat': 35.4375198

{'lat': 48.7129143, 'lng': 2.3629838}
{'lat': 38.7222524, 'lng': -9.1393366}
{'lat': 52.51604000000003, 'lng': 13.376910000000066}
{'lat': 13.3387613, 'lng': -81.3729372}
{'lat': 41.7735409, 'lng': 12.2397117}
{'lat': 40.4260459, 'lng': -3.5651646}
{'lat': 40.7830603, 'lng': -73.9712488}
{'lat': 42.24059889999999, 'lng': -8.7207268}
{'lat': 50.503887, 'lng': 4.469936}
{'lat': 28.500821, 'lng': -13.8628367}
{'lat': 59.38292819999999, 'lng': 17.9644707}
{'lat': 43.0097384, 'lng': -7.5567582}
{'lat': -34.7206336, 'lng': -58.25460510000001}
{'lat': 42.4298846, 'lng': -8.6446202}
{'lat': 39.0200099, 'lng': 1.4821482}
{'lat': 20.6596988, 'lng': -103.3496092}
{'lat': 33.886917, 'lng': 9.537499}
{'lat': 43.569240000000036, 'lng': -6.942619999999977}
{'lat': 40.707428, 'lng': -73.922845}
{'lat': -36.8201352, 'lng': -73.0443904}
{'lat': 53.4807593, 'lng': -2.2426305}
{'lat': 19.3437444, 'lng': -99.1561883}
{'lat': 51.2562128, 'lng': 7.150763599999999}
{'lat': 41.4387219, 'lng': 2.0126276}
{'lat'

In [12]:
usrloc = {}

for loc in df[df['user_timezone'].notnull()]['user_timezone'].unique():
    g = geocoder.google(loc.split(',')[0])
    latlng = g.latlng
    
    if latlng is None:
        g = geocoder.arcgis(loc.split(',')[0])
        latlng = g.latlng

        if latlng is None:
            g = geocoder.geocodefarm(loc.split(',')[0])
            latlng = g.latlng

            if latlng is None:
                g = geocoder.osm(loc.split(',')[0])
                latlng = g.latlng

                if latlng is None:
                    g = geocoder.yandex(loc.split(',')[0])
                    latlng = g.latlng
    
    try:
        usrloc[loc] = {'lat': latlng[0], 'lng': latlng[1]}
        print(usrloc[loc])
        
    except Exception as e:
        print(e)

{'lat': 53.3498053, 'lng': -6.2603097}
{'lat': 37.09024, 'lng': -95.712891}
{'lat': 37.51647579999999, 'lng': -82.8067126}
{'lat': 24.8607343, 'lng': 67.0011364}
{'lat': 38.7614031, 'lng': -120.5072407}
{'lat': 40.4167754, 'lng': -3.7037902}
{'lat': 19.4326077, 'lng': -99.133208}
{'lat': 10.488010000000031, 'lng': -66.87918999999994}
{'lat': 37.97614000000004, 'lng': 23.73640000000006}
{'lat': -34.60848999999996, 'lng': -58.37343999999996}
{'lat': -33.4488897, 'lng': -70.6692655}
{'lat': 71.706936, 'lng': -42.604303}
{'lat': 52.3702157, 'lng': 4.895167900000001}
{'lat': 52.52000659999999, 'lng': 13.404954}
{'lat': 45.807240000000036, 'lng': 15.967570000000023}
{'lat': 4.614960000000053, 'lng': -74.06940999999995}
{'lat': 48.856614, 'lng': 2.3522219}
{'lat': 51.5073509, 'lng': -0.1277583}
{'lat': 60.16985569999999, 'lng': 24.9383791}
{'lat': 28.2072168, 'lng': -177.3734926}
{'lat': 46.05062000000004, 'lng': 14.502820000000042}
{'lat': -34.6036844, 'lng': -58.3815591}
{'lat': 37.09024, '

In [13]:
df['tweet_location_coord'] = df['tweet_location'].map(ttloc)
df['user_timezone_coord'] = df['user_timezone'].map(usrloc)

In [14]:
df['tweet_location_coord'] = df['tweet_location_coord'].astype('object')
df['user_timezone_coord'] = df['user_timezone_coord'].astype('object')

In [15]:
df['tt_lat'] = df['tweet_location_coord'].apply(lambda x: None if type(x) is float else x['lat'])
df['tt_lng'] = df['tweet_location_coord'].apply(lambda x: None if type(x) is float else x['lng'])

In [16]:
df['usr_lat'] = df['user_timezone_coord'].apply(lambda x: None if type(x) is float else x['lat'])
df['usr_lng'] = df['user_timezone_coord'].apply(lambda x: None if type(x) is float else x['lng'])

In [17]:
df['hour'] = df['tweet_created'].dt.hour
df['date'] = df['tweet_created'].dt.date
df['dow'] = df['tweet_created'].dt.dayofweek

In [18]:
df['airline'] = df['text'].apply(lambda x: getAirlines(x))

In [28]:
df['tweet_id'] = df['tweet_id'].astype('int64')

In [29]:
df.head()

,airline_sentiment,is_reply,reply_count,retweet_count,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone,...,airline,tweet_location_coord,user_timezone_coord,tt_lat,tt_lng,usr_lat,usr_lng,hour,date,dow
0,positive,False,0,0,"""Los pilotos de Ryanair desconvocan la huelga ...",nan,2017-12-18 13:07:04,942743012337122944,NaN,Dublin,...,ryanair,NaN,"{'lat': 53.3498053, 'lng': -6.2603097}",nan,nan,53.349805,-6.260310,13,2017-12-18,0
1,positive,True,0,0,"@Iberia @lavecinarubia Si ,por favor las decla...",nan,2017-11-04 17:05:11,926857871916184064,NaN,NaN,...,iberia,NaN,NaN,nan,nan,nan,nan,17,2017-11-04,5
2,neutral,True,0,0,@Iberia Me dirías por favor que costo tiene?,nan,2017-12-02 15:24:09,936979305720091008,NaN,NaN,...,iberia,NaN,NaN,nan,nan,nan,nan,15,2017-12-02,5
3,negative,True,0,0,"@SupermanlopezN @Iberia @giroditalia Champion,...",nan,2017-12-21 23:17:43,943983853802328064,NaN,Central Time (US & Canada),...,iberia,NaN,"{'lat': 37.09024, 'lng': -95.712891}",nan,nan,37.090240,-95.712891,23,2017-12-21,3
4,negative,True,0,0,@SrtaFarrellDM @KLM @Iberia Eso de avianca es ...,nan,2017-12-06 00:44:25,938207464457212032,NaN,Eastern Time (US & Canada),...,klm,NaN,"{'lat': 37.51647579999999, 'lng': -82.8067126}",nan,nan,37.516476,-82.806713,0,2017-12-06,2


In [30]:
dfExport = df[['tweet_id','airline_sentiment', 'airline', 'lat', 'lon', 'tweet_location', 'tt_lat', 'tt_lng','user_timezone', 'usr_lat', 'usr_lng','hour','date','dow']]

In [31]:
dfExport.to_csv(path_or_buf=fileOut, sep=',', na_rep='', header=True, index=False, encoding='utf-8', mode='w', quotechar='"', line_terminator='\n', decimal='.')